In [ ]:
import pandas as pd
from pathlib import Path
import os
import json

In [ ]:
basepath = Path("/home/unix/wangyanz/codon_usage/scRNA")
data_pool = basepath.joinpath("data")

In [ ]:
## may take 25 mins

import os
import json
from pathlib import Path
import pandas as pd
from tqdm import tqdm

table = {
        "TTT": "F", "TTC": "F", "TTA": "L", "TTG": "L",
        "TCT": "S", "TCC": "S", "TCA": "S", "TCG": "S",
        "TAT": "Y", "TAC": "Y",                           # noqa: E241
        "TGT": "C", "TGC": "C",             "TGG": "W",   # noqa: E241
        "CTT": "L", "CTC": "L", "CTA": "L", "CTG": "L",
        "CCT": "P", "CCC": "P", "CCA": "P", "CCG": "P",
        "CAT": "H", "CAC": "H", "CAA": "Q", "CAG": "Q",
        "CGT": "R", "CGC": "R", "CGA": "R", "CGG": "R",
        "ATT": "I", "ATC": "I", "ATA": "I", "ATG": "M",
        "ACT": "T", "ACC": "T", "ACA": "T", "ACG": "T",
        "AAT": "N", "AAC": "N", "AAA": "K", "AAG": "K",
        "AGT": "S", "AGC": "S", "AGA": "R", "AGG": "R",
        "GTT": "V", "GTC": "V", "GTA": "V", "GTG": "V",
        "GCT": "A", "GCC": "A", "GCA": "A", "GCG": "A",
        "GAT": "D", "GAC": "D", "GAA": "E", "GAG": "E",
        "GGT": "G", "GGC": "G", "GGA": "G", "GGG": "G",
    }
stop_codons=["TAA", "TAG", "TGA"]
start_codons=["TTG", "CTG", "ATG"]

df = pd.read_csv(data_pool.joinpath("cell_type_norm_mean_scRNA_Muris_10X_v2.csv"))
df = df.set_index('cell_type')

with open(data_pool.joinpath("gene_name_scRNA_Muris_10X_v2.csv"), "r") as fp:
    gene_name_scRNA = fp.read().split()

gene_seq_pool = Path("/home/unix/wangyanz/13.yanze/data/gene_seq")
gene_count_pool = Path("/home/unix/wangyanz/13.yanze/data/gene_count")
gene_cell_type_pool = data_pool.joinpath("cell_type")

# available_gene_seq = gene_seq_pool.glob("*.dat")
cell_types = df.index.tolist()

for cell_type in tqdm(cell_types, desc="cell type", position=0):
    print(cell_type)
    empty_data = {}
    for codon in table:
        empty_data[codon] = 0
    for stop_codon in stop_codons:
        empty_data[stop_codon] = 0

    query_cell_type = cell_type
    query_gene_expr = df.loc[query_cell_type]
    count = 0

    available_gene_seq = gene_seq_pool.glob("*.dat")

    for agene in tqdm(available_gene_seq, desc="gene", position=1):
        agene_index = agene.stem
        gene_name = agene_index.split("_")[0]
        if gene_name in gene_name_scRNA:
            gene_count_file = gene_count_pool.joinpath(f"{agene_index}.json")
            with open(gene_count_file, "r") as fp:
                data = json.load(fp)
            for key in data.keys():
                empty_data[key] += query_gene_expr[gene_name] * data[key]

    if not os.path.exists(gene_cell_type_pool):
        os.mkdir(gene_cell_type_pool)

    canonical_cell_name = "_".join(query_cell_type.split())
    with open(gene_cell_type_pool.joinpath(f"{canonical_cell_name}.json"), "w") as fp:
        json.dump(empty_data, fp)


plot

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
cell_type_pool = data_pool.joinpath("cell_type")
pics_dir = data_pool.joinpath("pics")
if not os.path.exists(pics_dir):
    os.mkdir(pics_dir)

cells = cell_type_pool.glob("*.json")
for cell in cells:
    with open(cell, "r") as fp:
        data = json.load(fp)
    myarray = np.array(list(data.values()))
    # print(myarray)
    myarray = myarray / np.sum(myarray)
    plt.figure()
    plt.xlabel("codons", )
    plt.ylabel("usage frequency")
    plt.title(f"{cell.stem} codon usage")
    plt.xticks(rotation=90, size=5)
    plt.bar(data.keys(), myarray, alpha=0.5, label=cell)
    plt.savefig(pics_dir.joinpath(f"{cell.stem}.png"))
    plt.close()
